In [81]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup as bs
import re

In [89]:
def take_infos(driver,noms,localis,numeros,gpss):
    phars=driver.find_elements(By.CLASS_NAME,'classic-view-grids-content-area')
    for infos in phars:
        try:
            soup=bs(infos.get_attribute('innerHTML'))

            nom=soup.find('h4',class_='lp-h4').text
            nom=re.sub(r'\s{1,}', ' ', nom)
            
            local=soup.find('div',class_='show').text
            local=re.sub(r'\s{1,}', ' ', local)
           
            numero=soup.find('a',class_='grind-number')
            if numero:
                numeros.append(numero.text)
            else:
                numeros.append('+00')

            gps=soup.find('a',class_='show-loop-map-popup')
            if gps:
                lat=gps.get('data-lat')
                lng=gps.get('data-lng')
                gpss.append(lat+" :: "+lng)
            else:
                gpss.append(" :: ")
            
            localis.append(local)
            noms.append(nom)
        except:
            continue
    

In [90]:
def pharmacie():
    driver = webdriver.Chrome()
    driver.get('https://pagesjaunes.ci/')
    sleep(5)
    driver.find_element(By.ID,'select').send_keys('Pharmacie')
    driver.find_element(By.CLASS_NAME,'lp-search-btn').submit()
    sleep(5)
    numeros=[]
    noms=[]
    localis=[]
    gpss=[]
    take_infos(driver,noms,localis,numeros,gpss)
    driver.find_element(By.CSS_SELECTOR,'#lp-pages-in-cats > div > ul > li:nth-child(1)').click()
    sleep(5)
    for i in range(2,7):
        sleep(5)
        take_infos(driver,noms,localis,numeros,gpss)
        
    for i in range(7,13):
        i+=1
        driver.find_element(By.CSS_SELECTOR,f'#content-grids > div.lp-pagination.pagination.lp-filter-pagination-ajx > ul > li:nth-child({i})').click()
        sleep(5)
        take_infos(driver,noms,localis,numeros,gpss)
    
    for i in range(80):
        driver.find_element(By.CSS_SELECTOR,f'#content-grids > div.lp-pagination.pagination.lp-filter-pagination-ajx > ul > li:nth-child(13)').click()
        sleep(5)
        take_infos(driver,noms,localis,numeros,gpss)
        
    data = {'numoros': numeros, 'nom': noms,'localisation': localis,'cordonnées gps': gpss}
    dt = pd.DataFrame(data)
    dt.to_excel('pharmacie.xlsx')

In [91]:
pharmacie()